**Simple example for Asg1 with handcrafted feature**

An example diagram for instance search.
Please read the slide "Information for Asg1" first.

In every bounding box file (the txt files), each line records the coordinates of one bounding box in format: x of top-left point, y of top-left point, width, height.

The bounding box information is only provided for the query images.

Notes: it's possible that there are more than one instances in a query image (see the annotation(s) in the corresponding txt file), you should find the most similar image rank list in the gallery by considering all the instances.

In [1]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 
#import matplotlib.pyplot as plt
from tqdm import tqdm
import threading


 Download the dataset from this linke [onedrive link](https://portland-my.sharepoint.com/:u:/g/personal/srwang3-c_my_cityu_edu_hk/EZ0BIZatMIJMoiCG4-uy6okBSlXLJD7TUyIDG1lbKUJ0eA?e=lbKz4a).
And unzip the downloaded file into some path. 
> In this tutorial, the path is `/Users/txsing/datasets_4186`, please replace to your own accordingly.

Initialize the necessary parameters, including paths, feature extractors.

In [2]:
download_path='D:/Users/KLIVANCHAN/Desktop/CityU' # change to your own download path
path_query=download_path+'/query_4186'
path_query_txt=download_path+'/query_txt_4186'

# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)

# sift = cv2.ORB_create()
# bf = cv2.BFMatcher()

# FLANN_INDEX_KDTREE = 1
# index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
# # search_params = dict(checks=50)
# flann = cv2.FlannBasedMatcher(index_params,{})

record_all=np.zeros((num_query,len(name_gallery)))

query_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_query+'/*.jpg')]
query_imgs_no = [x[:-4] for x in query_imgs_no]

gallery_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_gallery+'/*.jpg')]
gallery_imgs_no = [x[:-4] for x in gallery_imgs_no]

Process

Feature extraction -> calculate distance -> compare

In [ ]:
# # the iteration loop for query 
# # iteration times is 2 here, only for the demonstration
# for i, query_img_no in enumerate(query_imgs_no[0:1]):
    
#     time_s = time.time()
#     dist_record=[]
#     per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
#     per_query=cv2.imread(per_query_name)
#     per_query = cv2.cvtColor(per_query, cv2.COLOR_BGR2GRAY)
#     gallery_imgs_no_desc=[]
    
#     # read boundary from text file
#     queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
#     boundary_file = open(queryfilename, 'r')
#     boundary = boundary_file.readline().strip().split(' ')
#     boundary = [int(b) for b in boundary]
#     boundary_file.close()
    
#     # crop the image
#     x ,y, w, h = boundary
#     query_boundary = per_query[y:y+h, x:x+w]
    
#     # feature extraction for per query
#     # the bounding box information is not considered
#     # quite naive, just an example
#     per_query_kp, per_query_des = sift.detectAndCompute(query_boundary,None)
    
#     # the iteration loop for gallery
#     for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
#         per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
#         per_gallery=cv2.imread(per_gallery_name)
#         per_gallery = cv2.cvtColor(per_gallery, cv2.COLOR_BGR2GRAY)
#         # feature extraction for per gallery
#         per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery,None)
        
#         # use part of the features to make the calculation feasible
#         # quite naive, just an example
#         matches = bf.knnMatch(np.asarray(per_query_des,np.float32),np.asarray(per_gallery_des,np.float32),k=2)
# #         matches = flann.knnMatch(per_query_des.astype(np.float32),per_gallery_des.astype(np.float32),k=2)

#         # Compute similarity score for each match
#         sim_score = len(matches) / (np.sqrt(len(per_query_des)) * np.sqrt(len(per_gallery_des)))
#         dist_record.append(sim_score)
# #         cv2.imshow('result',per_result_img)
# #         cv2.waitKey(0)
# #         cv2.destroyAllWindows()

#     # find the indexes with descending similarity order
#     ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
#     # update the results for one query
#     for k in range(len(ascend_index)):
#         gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[ascend_index[k]])
#     record_all[i,:]= gallery_imgs_no_desc
#     time_e = time.time()
#     sorted(dist_record)
#     print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
#     query_idx = i
#     print(f'For query image No. {query_imgs_no[query_idx]}, the top ranked similar image No. is {gallery_imgs_no_desc[0]}.')
# instance search function
def instance_search(query_img_no,per_gallery,i,progress_bar):

    sift = cv2.ORB_create()
#     sift = cv2.SIFT_create()
#     sift = cv2.xfeatures2d.SURF_create()
    bf = cv2.BFMatcher()

#     FLANN_INDEX_KDTREE = 1
#     index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
#     search_params = dict(checks=50)
#     flann = cv2.FlannBasedMatcher(index_params,dict(checks=50))
    
    time_s = time.time()
    dist_record=[]
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    per_query = cv2.cvtColor(per_query, cv2.COLOR_BGR2GRAY)
    gallery_imgs_no_desc=[]
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    boundary_file = open(queryfilename, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()

    # crop the image
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]
#     cv2.imshow('result',query_boundary)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    # feature extraction for per query
    # the bounding box information is not considered
    # quite naive, just an example
    per_query_kp, per_query_des = sift.detectAndCompute(query_boundary,None)
    # the iteration loop for gallery
    for j, gallery_img_no in enumerate(gallery_imgs_no):
#         per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
#         per_gallery=cv2.imread(per_gallery_name)
#         per_gallery = cv2.cvtColor(per_gallery, cv2.COLOR_BGR2GRAY)
        # feature extraction for per gallery
        per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery[j],None)
        
        # use part of the features to make the calculation feasible
        # quite naive, just an example
        matches = bf.knnMatch(per_query_des,per_gallery_des,k=2)
#         matches = flann.knnMatch(per_query_des.astype(np.float32),per_gallery_des.astype(np.float32),k=2)
    
        # Compute similarity score for each match
        sim_score = len(matches) / (np.sqrt(len(per_query_des)) * np.sqrt(len(per_gallery_des)))

        dist_record.append(sim_score)
#         print(f"\rThread {str(i)}: {str(format((j/5000)*100,'.2f'))}%  ",end = "\r")
#         cv2.imshow('result',per_result_img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        progress_bar.update(1)

    # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
    # update the results for one query
    for k in range(len(ascend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[ascend_index[k]])
    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    sorted(dist_record)
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top ranked similar image No. is {gallery_imgs_no_desc[0]}.')
    
    
per_gallery = [0 for i in range(5000)]
for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no)):
        per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
        per_gallery[j] =cv2.imread(per_gallery_name)
        per_gallery[j] = cv2.cvtColor(per_gallery[j], cv2.COLOR_BGR2GRAY)
# Create a list of threads
threads = []
for i, query_img_no in tqdm(enumerate(query_imgs_no[0:20])):
    progress_bar = tqdm(total=num_gallery, desc=f"Processing query {i+1}/{num_query}")
    thread = threading.Thread(target=instance_search, args=(query_imgs_no[i], per_gallery, i, progress_bar))
    threads.append(thread)
    
# Start all threads
for thread in threads:
    thread.start()
#     thread.join()

# Wait for all threads to finish
for thread in threads:
    thread.join()
    

5000it [02:50, 29.33it/s]
0it [00:00, ?it/s]
Processing query 1/20:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing query 2/20:   0%|          | 0/5000 [00:00<?, ?it/s]


Processing query 3/20:   0%|          | 0/5000 [00:00<?, ?it/s]



4it [00:00, 20.30it/s]   0%|          | 0/5000 [00:00<?, ?it/s]




Processing query 5/20:   0%|          | 0/5000 [00:00<?, ?it/s]





Processing query 6/20:   0%|          | 0/5000 [00:00<?, ?it/s]






Processing query 7/20:   0%|          | 0/5000 [00:00<?, ?it/s]







Processing query 8/20:   0%|          | 0/5000 [00:00<?, ?it/s]








Processing query 9/20:   0%|          | 0/5000 [00:00<?, ?it/s]









Processing query 10/20:   0%|          | 0/5000 [00:00<?, ?it/s]










Processing query 11/20:   0%|          | 0/5000 [00:00<?, ?it/s]











Processing query 12/20:   0%|          | 0/5000 [00:00<?, ?it/s]












Processing query 13/20:   0%|          | 0/5000 [00:00<?, ?it/s]













Processing query 14/20:   0%

Processing query 13/20:   0%|          | 4/5000 [00:07<1:39:26,  1.19s/it]





Processing query 6/20:   0%|          | 4/5000 [00:07<1:41:26,  1.22s/it]











Processing query 12/20:   0%|          | 4/5000 [00:07<1:37:26,  1.17s/it]














Processing query 15/20:   0%|          | 4/5000 [00:07<1:37:21,  1.17s/it]






Processing query 7/20:   0%|          | 4/5000 [00:07<1:46:25,  1.28s/it]







Processing query 8/20:   0%|          | 4/5000 [00:07<1:38:45,  1.19s/it]















Processing query 1/20:   0%|          | 4/5000 [00:07<1:40:59,  1.21s/it]










Processing query 11/20:   0%|          | 4/5000 [00:07<1:38:10,  1.18s/it]








Processing query 9/20:   0%|          | 4/5000 [00:07<1:39:57,  1.20s/it]
















Processing query 17/20:   0%|          | 4/5000 [00:07<1:42:43,  1.23s/it]




Processing query 5/20:   0%|          | 4/5000 [00:07<1:37:07,  1.17s/it]













Processing query 14/20:   0%|          | 4/5000 [00:07<1:47:40,  1.29s/i

Processing query 12/20:   0%|          | 10/5000 [00:08<22:21,  3.72it/s]



Processing query 4/20:   0%|          | 10/5000 [00:08<21:15,  3.91it/s]








Processing query 9/20:   0%|          | 10/5000 [00:08<23:03,  3.61it/s]









Processing query 10/20:   0%|          | 10/5000 [00:08<21:48,  3.81it/s]





Processing query 6/20:   0%|          | 10/5000 [00:08<22:26,  3.71it/s]

Processing query 2/20:   0%|          | 10/5000 [00:08<22:49,  3.64it/s]














Processing query 15/20:   0%|          | 10/5000 [00:08<23:09,  3.59it/s]
















Processing query 17/20:   0%|          | 10/5000 [00:08<22:56,  3.63it/s]












Processing query 19/20:   0%|          | 10/5000 [00:08<21:49,  3.81it/s]















Processing query 16/20:   0%|          | 11/5000 [00:08<18:47,  4.42it/s]


Processing query 1/20:   0%|          | 11/5000 [00:08<20:30,  4.05it/s]




Processing query 5/20:   0%|          | 11/5000 [00:08<22:26,  3.70it/s]







Processing query 8/20:

Processing query 9/20:   0%|          | 17/5000 [00:09<09:25,  8.81it/s]



Processing query 4/20:   0%|          | 18/5000 [00:09<09:17,  8.94it/s]












Processing query 13/20:   0%|          | 17/5000 [00:08<10:27,  7.94it/s]







Processing query 8/20:   0%|          | 16/5000 [00:09<12:14,  6.79it/s]




Processing query 5/20:   0%|          | 17/5000 [00:09<11:23,  7.29it/s]





Processing query 6/20:   0%|          | 19/5000 [00:09<08:52,  9.35it/s]

Processing query 2/20:   0%|          | 17/5000 [00:09<10:59,  7.56it/s]















Processing query 16/20:   0%|          | 18/5000 [00:08<09:07,  9.10it/s]


Processing query 3/20:   0%|          | 18/5000 [00:09<09:54,  8.37it/s]










Processing query 11/20:   0%|          | 19/5000 [00:09<08:16, 10.03it/s]

















Processing query 1/20:   0%|          | 17/5000 [00:09<11:05,  7.49it/s]






Processing query 19/20:   0%|          | 19/5000 [00:09<08:15, 10.05it/s][A




Processing query 5/20:   0%|    

Processing query 12/20:   1%|          | 26/5000 [00:09<06:37, 12.52it/s]













Processing query 14/20:   1%|          | 26/5000 [00:09<07:01, 11.79it/s]

Processing query 2/20:   0%|          | 24/5000 [00:09<07:05, 11.70it/s]





Processing query 6/20:   0%|          | 25/5000 [00:09<08:00, 10.36it/s]







Processing query 8/20:   0%|          | 24/5000 [00:09<06:39, 12.45it/s]


Processing query 1/20:   1%|          | 26/5000 [00:09<06:16, 13.23it/s]












Processing query 19/20:   1%|          | 27/5000 [00:09<06:23, 12.97it/s]













Processing query 14/20:   1%|          | 28/5000 [00:09<06:17, 13.16it/s]











Processing query 12/20:   1%|          | 28/5000 [00:09<06:15, 13.22it/s]
















Processing query 17/20:   1%|          | 28/5000 [00:09<06:36, 12.53it/s]










Processing query 11/20:   1%|          | 28/5000 [00:09<07:22, 11.23it/s]















Processing query 16/20:   1%|          | 28/5000 [00:09<06:38, 12.47it/s]






Proce

...

Processing query 13/20:   4%|▎         | 182/5000 [00:31<08:38,  9.30it/s]

















Processing query 18/20:   4%|▎         | 182/5000 [00:31<08:49,  9.09it/s]













Processing query 14/20:   4%|▎         | 182/5000 [00:31<09:34,  8.39it/s]















Processing query 16/20:   4%|▎         | 182/5000 [00:31<09:18,  8.63it/s]



Processing query 4/20:   4%|▎         | 182/5000 [00:31<09:03,  8.86it/s]














Processing query 15/20:   4%|▎         | 182/5000 [00:31<09:02,  8.89it/s]








Processing query 9/20:   4%|▎         | 182/5000 [00:31<09:06,  8.81it/s]
















Processing query 17/20:   4%|▎         | 182/5000 [00:31<09:02,  8.87it/s]

Processing query 2/20:   4%|▎         | 182/5000 [00:31<08:59,  8.93it/s]









Processing query 1/20:   4%|▎         | 183/5000 [00:31<08:32,  9.40it/s]




Processing query 5/20:   4%|▎         | 183/5000 [00:31<08:19,  9.64it/s]


Processing query 3/20:   4%|▎         | 183/5000 [00:31<08:47,  9.14it/s]

Pro

Processing query 9/20:   4%|▍         | 188/5000 [00:31<09:57,  8.05it/s]

Processing query 2/20:   4%|▍         | 188/5000 [00:32<09:58,  8.04it/s]





Processing query 19/20:   4%|▍         | 188/5000 [00:31<09:57,  8.05it/s][A
















Processing query 17/20:   4%|▍         | 189/5000 [00:31<08:58,  8.93it/s]













Processing query 14/20:   4%|▍         | 189/5000 [00:31<09:14,  8.67it/s]






Processing query 7/20:   4%|▍         | 189/5000 [00:31<09:28,  8.46it/s]


Processing query 3/20:   4%|▍         | 189/5000 [00:32<09:09,  8.75it/s]









Processing query 10/20:   4%|▍         | 189/5000 [00:31<09:20,  8.58it/s]










Processing query 11/20:   4%|▍         | 189/5000 [00:32<09:37,  8.33it/s]














Processing query 15/20:   4%|▍         | 189/5000 [00:31<09:56,  8.06it/s]



Processing query 4/20:   4%|▍         | 189/5000 [00:32<10:00,  8.01it/s]

Processing query 2/20:   4%|▍         | 189/5000 [00:32<10:04,  7.95it/s]

















Process

Processing query 3/20:   4%|▍         | 195/5000 [00:33<16:00,  5.00it/s]













Processing query 14/20:   4%|▍         | 195/5000 [00:33<16:33,  4.84it/s]














Processing query 15/20:   4%|▍         | 195/5000 [00:33<17:25,  4.60it/s]





Processing query 6/20:   4%|▍         | 195/5000 [00:33<17:49,  4.49it/s]



Processing query 4/20:   4%|▍         | 195/5000 [00:33<17:06,  4.68it/s]












Processing query 13/20:   4%|▍         | 195/5000 [00:33<16:39,  4.81it/s]






Processing query 19/20:   4%|▍         | 195/5000 [00:33<17:51,  4.49it/s][A









Processing query 10/20:   4%|▍         | 195/5000 [00:33<16:57,  4.72it/s]




Processing query 5/20:   4%|▍         | 195/5000 [00:33<16:55,  4.73it/s]











Processing query 12/20:   4%|▍         | 195/5000 [00:33<17:12,  4.65it/s]








Processing query 9/20:   4%|▍         | 195/5000 [00:33<18:16,  4.38it/s]







Processing query 8/20:   4%|▍         | 195/5000 [00:33<19:36,  4.08it/s]














Processing query 3/20:   4%|▍         | 200/5000 [00:34<14:03,  5.69it/s]







Processing query 8/20:   4%|▍         | 200/5000 [00:34<14:20,  5.58it/s]












Processing query 13/20:   4%|▍         | 200/5000 [00:34<13:48,  5.79it/s]











Processing query 12/20:   4%|▍         | 200/5000 [00:34<14:22,  5.57it/s]















Processing query 16/20:   4%|▍         | 200/5000 [00:34<14:14,  5.62it/s]

Processing query 2/20:   4%|▍         | 200/5000 [00:34<14:23,  5.56it/s]

















Processing query 18/20:   4%|▍         | 200/5000 [00:34<14:07,  5.66it/s]



Processing query 4/20:   4%|▍         | 200/5000 [00:34<15:28,  5.17it/s]









Processing query 10/20:   4%|▍         | 200/5000 [00:34<15:31,  5.15it/s]














Processing query 15/20:   4%|▍         | 201/5000 [00:34<12:46,  6.26it/s]










Processing query 11/20:   4%|▍         | 200/5000 [00:34<15:25,  5.19it/s]





Processing query 6/20:   4%|▍         | 200/5000 [00:34<16:16,  4.92it/s]



Processing query 6/20:   4%|▍         | 205/5000 [00:36<33:46,  2.37it/s]
















Processing query 17/20:   4%|▍         | 205/5000 [00:36<33:40,  2.37it/s]


Processing query 3/20:   4%|▍         | 205/5000 [00:36<35:31,  2.25it/s]







Processing query 1/20:   4%|▍         | 205/5000 [00:36<37:52,  2.11it/s]














Processing query 15/20:   4%|▍         | 205/5000 [00:36<40:06,  1.99it/s]












Processing query 13/20:   4%|▍         | 205/5000 [00:36<34:42,  2.30it/s]










Processing query 11/20:   4%|▍         | 205/5000 [00:36<35:30,  2.25it/s]













Processing query 19/20:   4%|▍         | 205/5000 [00:36<34:50,  2.29it/s]








Processing query 9/20:   4%|▍         | 205/5000 [00:36<36:48,  2.17it/s]

Processing query 2/20:   4%|▍         | 205/5000 [00:36<36:33,  2.19it/s]






Processing query 7/20:   4%|▍         | 205/5000 [00:36<40:13,  1.99it/s]



Processing query 4/20:   4%|▍         | 205/5000 [00:36<36:20,  2.20it/s]

















Output 

Save the rank list file.

In [ ]:
# write the output file following the example
f=open(r'./rank_list.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()